In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import namedtuple, deque

### Function used to reset training weights

In [2]:
'''
This function is used in the reset_parameters function.
It helps to generate a random weight matrix to start training.
'''
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

# Actor NN

In [3]:
class Actor(nn.Module):

    def __init__(self, state_size, action_size, seed, fc1_units=256, fc2_units=128):
        super(Actor, self).__init__()
        
        #Set the random # seed for reproducibility
        self.seed = torch.manual_seed(seed)
        
        #Layer 1  (input is vector state therefore input size is state_size)
        self.fc1 = nn.Linear(state_size, fc1_units)      
        
        #Layer 2
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        
        #Output layer (output is vector action therefore output size is action_size)
        self.fc3 = nn.Linear(fc2_units, action_size)
        
        #Randomly initialize weight matrices
        self.reset_parameters()

    #To randomly initialize weight matrices    
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
    
    #Forward propagation: maps the state (input) to actions (output)
    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))

# Critic NN

In [4]:
class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=256, fc2_units=128):
        super(Critic, self).__init__()

        #Set the random # seed for reproducibility
        self.seed = torch.manual_seed(seed)
        
        #Layer 1
        self.fcs1 = nn.Linear(state_size, fcs1_units)
        
        #Layer 2
        self.fc2 = nn.Linear(fcs1_units+action_size, fc2_units)

        #Output layer (output is Q value therefore size is 1)
        self.fc3 = nn.Linear(fc2_units, 1)
        
        #Randomly initialize weight matrices
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Memory buffer

In [ ]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        
        # internal memory (deque). Deque creates a list but allows to append elements to begginning or end
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        
        #Saved as namedtupple instead of list to make it easier to access specific values (positions) later on.
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        
        #Again, by using a named tupple now we can acess the action of this experience by calling e.action
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
        
        #Find all states in memory batch, append vertically, convert to torch with float values and send to CPU or GPU
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        
        #Repeat previous line but with actions, rewards, next states and dones
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        #Return the batch of tensors
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)